In [1]:
import yaml
from pathlib import Path
import numpy as np
from ase.build.molecule import molecule
from ase.calculators.emt import EMT
from ase.md.verlet import VelocityVerlet
from ase import units as u

In [2]:
h2o = molecule('H2O')
h2o.positions[0, 0] += .05
h2o.positions

array([[ 0.05    ,  0.      ,  0.119262],
       [ 0.      ,  0.763239, -0.477047],
       [ 0.      , -0.763239, -0.477047]])

In [3]:
def md2dict(obj):
    dct = {
        'nsteps': obj.nsteps,
        'dt': obj.dt,
        'masses': obj.masses.T.tolist()[0]
    }
    return dct

In [4]:
def atoms2dict(atoms):
    atoms_dict = {
        'positions': atoms.positions.tolist()
    }
    
    if any(atoms.pbc):
        atoms_dict['cell'] = atoms.cell.tolist()
    
    return atoms_dict

In [5]:
def calc2dict(calc):    
    calc_dict = {}
    
    # convert numpy arrays into ordinary lists
    for key, val in calc.results.items():
        if isinstance(val, np.ndarray):
            calc_dict[key] = val.tolist()
        else:
            calc_dict[key] = val
    
    return calc_dict

In [6]:
def step2dict(atoms, md):
    md_dict = md2dict(md)
    
    atoms_dict = atoms2dict(atoms)
    calc_dict = calc2dict(atoms.calc)
    
    return {'MD': md_dict, 'atoms': atoms_dict, 'calculator': calc_dict}

In [7]:
def metadata2dict(atoms, calc, md):
    md_dict = md.todict()
    md_dict['fs'] = u.fs
    
    atoms_dict = atoms2dict(atoms)
    atoms_dict['numbers'] = atoms.numbers.tolist()
    atoms_dict['symbols'] = [f'{sym}' for sym in atoms.symbols]
    
    calc_dict = {
        'name': calc.__class__.__name__,
        'params': calc.todict()
    }
    
    meta_dict = {
        'atoms': atoms_dict,
        'md': md_dict,
        'calculator': calc_dict,
    }
    
    return meta_dict

In [8]:
def to_yaml(obj, file, mode='a'):
    with open(file, mode) as f:
        yaml.dump(obj, f)

### Initialize MD and calculator

In [9]:
h2o.calc = EMT()
md = VelocityVerlet(h2o, timestep=u.fs)

### Save metadata and initialize trajectory

In [10]:
metadata = metadata2dict(h2o, h2o.calc, md)
metadata

{'atoms': {'positions': [[0.05, 0.0, 0.119262],
   [0.0, 0.763239, -0.477047],
   [0.0, -0.763239, -0.477047]],
  'numbers': [8, 1, 1],
  'symbols': ['O', 'H', 'H']},
 'md': {'type': 'molecular-dynamics',
  'md-type': 'VelocityVerlet',
  'timestep': 0.09822694788464063,
  'fs': 0.09822694788464063},
 'calculator': {'name': 'EMT', 'params': {}}}

In [11]:
to_yaml(metadata, 'md_metadata.yaml', mode='w')

trajectory = Path('trajectory.yaml')
if trajectory.exists():
    trajectory.unlink()    

### Run MD

In [12]:
for _ in range(2):
    md.run(1)
    to_yaml([step2dict(h2o, md)], trajectory)

In [13]:
with open(trajectory) as f:
    traj = yaml.load(f)

In [14]:
traj

[{'MD': {'dt': 0.09822694788464063,
   'masses': [15.999, 1.008, 1.008],
   'nsteps': 1},
  'atoms': {'positions': [[0.05019278480688103, 0.0, 0.12156118630812841],
    [-0.0015299425224650789, 0.7875231843281152, -0.49529336991257256],
    [-0.0015299425224650789, -0.7875231843281152, -0.49529336991257256]]},
  'calculator': {'energy': 2.264562056808295,
   'forces': [[0.4409496308260477, 0.0, 5.258844668164993],
    [-0.22047481541302386, 3.473854359352102, -2.6294223340824967],
    [-0.22047481541302386, -3.473854359352102, -2.6294223340824967]],
   'free_energy': 2.264562056808295}},
 {'MD': {'dt': 0.09822694788464063,
   'masses': [15.999, 1.008, 1.008],
   'nsteps': 2},
  'atoms': {'positions': [[0.050651493558543675, 0.0, 0.12703182944803243],
    [-0.005170260636478305, 0.8450589553956777, -0.5387084589975548],
    [-0.005170260636478305, -0.8450589553956777, -0.5387084589975548]]},
  'calculator': {'energy': 1.8950135355994473,
   'forces': [[0.07896134196623283, 0.0, 0.941707